In [ ]:
import os
import glob
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager # 确保字体管理器已导入
from tqdm.autonotebook import tqdm # 用于显示进度


plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# --- 配置路径 ---
val_image_dir=      r'f:\val_img'
output_mask_dir=    r'result\result-4-14\Segmentation_Results'

# 1. 原始验证集图像路径
#    (使用之前代码块中定义的 val_image_dir)
if 'val_image_dir' not in locals() or not os.path.isdir(val_image_dir):
    print("错误：原始验证集图像路径 'val_image_dir' 未定义或无效。请确保它已在之前的单元格中设置。")
    # 你可能需要根据实际情况手动设置:
    # val_image_dir = "/kaggle/input/pterygium/val_img/val_img" # Kaggle 示例
    # val_image_dir = "f:/val" # 本地示例
else:
    print(f"使用原始验证图像路径: {val_image_dir}")

# 2. 预测掩码图像路径 (包含模型生成的 RGB 掩码)
#    (使用之前代码块中定义的 output_mask_dir)
if 'output_mask_dir' not in locals() or not os.path.isdir(output_mask_dir):
    print("错误：预测掩码路径 'output_mask_dir' 未定义或无效。请确保它已在之前的单元格中设置，并且包含了预测生成的掩码文件。")
    # 你可能需要根据实际情况手动设置:
    # output_mask_dir = "/kaggle/working/mask" # Kaggle 示例
    # output_mask_dir = "/content/mask" # Colab 示例
    # output_mask_dir = "f:/mask" # 本地示例
else:
    print(f"使用预测掩码路径: {output_mask_dir}")

# --- 可视化函数 ---
def visualize_overlay(original_img_path, mask_img_path):
    """加载原图和掩码，生成灰度覆盖图并显示"""
    try:
        # 加载原图
        original_image = Image.open(original_img_path).convert("RGB")
        # 加载预测掩码 (RGB)
        mask_image = Image.open(mask_img_path).convert("RGB")

        # 检查尺寸是否一致 (可选，但推荐)
        if original_image.size != mask_image.size:
            print(f"警告：图像 {os.path.basename(original_img_path)} ({original_image.size}) "
                f"和掩码 {os.path.basename(mask_img_path)} ({mask_image.size}) 尺寸不匹配，跳过可视化。")
            return

        # 1. 将原图转为灰度，再转回 RGB 以便合并
        grayscale_image = original_image.convert("L")
        grayscale_rgb_image = grayscale_image.convert("RGB")
        grayscale_rgb_array = np.array(grayscale_rgb_image)

        # 2. 获取掩码数组
        mask_array = np.array(mask_image)

        # 3. 创建覆盖图像数组 (初始化为灰度RGB)
        overlay_array = grayscale_rgb_array.copy()

        # 4. 找到掩码中红色区域 (R=128, G=0, B=0) 的条件
        #    为增加鲁棒性，可以只检查 R 通道是否 > 0 (或接近 128)
        mask_condition = mask_array[:, :, 0] > 64 # 条件放宽一点，>64 应该能捕捉到 128

        # 5. 将掩码区域覆盖到灰度图上
        overlay_array[mask_condition] = mask_array[mask_condition]

        # 6. 转回 PIL Image
        overlay_image = Image.fromarray(overlay_array)

        # 7. 可视化
        fig, axes = plt.subplots(1, 2, figsize=(12, 6)) # 调整图像大小

        # 显示灰度图
        axes[0].imshow(grayscale_rgb_image)
        axes[0].set_title(f"灰度原图: {os.path.basename(original_img_path)}")
        axes[0].axis('off')

        # 显示覆盖图
        axes[1].imshow(overlay_image)
        axes[1].set_title(f"预测掩码覆盖: {os.path.basename(mask_img_path)}")
        axes[1].axis('off')

        plt.tight_layout()
        plt.show()

    except FileNotFoundError:
        print(f"错误：文件未找到 - {original_img_path} 或 {mask_img_path}")
    except Exception as e:
        print(f"处理图像 {os.path.basename(original_img_path)} 时发生错误: {e}")


# --- 执行可视化 ---
# 检查路径是否有效
if 'val_image_dir' in locals() and os.path.isdir(val_image_dir) and \
        'output_mask_dir' in locals() and os.path.isdir(output_mask_dir):

    # 查找所有原始验证图像 (例如 .png, 根据你的文件类型修改)
    val_image_paths = sorted(glob.glob(os.path.join(val_image_dir, "*.png")))

    if not val_image_paths:
        print(f"错误：在目录 {val_image_dir} 中未找到任何 .png 图像文件。")
    else:
        print(f"找到 {len(val_image_paths)} 张验证图像，开始可视化...")

        # 遍历所有找到的验证图像
        for img_path in tqdm(val_image_paths, desc="可视化进度"):
            # 构建对应的掩码文件路径
            base_name = os.path.basename(img_path)
            mask_path = os.path.join(output_mask_dir, base_name) # 假设掩码文件名与原图名一致

            # 检查对应的掩码文件是否存在
            if os.path.exists(mask_path):
                # 调用可视化函数
                visualize_overlay(img_path, mask_path)
            else:
                print(f"警告：找不到对应的预测掩码文件 {mask_path}，跳过图像 {base_name}。")

        print("\n所有验证结果可视化完成。")
else:
    print("错误：无法执行可视化，因为 'val_image_dir' 或 'output_mask_dir' 无效。")